In [1]:
import pandas as pd
import numpy as np

In [2]:
data_a = pd.read_csv('arrh.csv')
data_a.head()

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,class
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,0
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,0
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,0


In [3]:
data_a.shape

(452, 280)

In [38]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

In [5]:
t_now = 400

In [9]:
data_past = data_a.iloc[:t_now,]
data_new = data_a.iloc[t_now:]
print(data_past.shape)
print(data_new.shape)

x_test, y_test = data_new.drop('class', 1), data_new['class']

(400, 280)
(52, 280)


C:\Users\Jerrold's work\AppData\Local\Temp\ipykernel_13832\3377173372.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_test, y_test = data_new.drop('class', 1), data_new['class']


In [23]:
x_train, y_train = data_past.iloc[:,:-1], data_past.iloc[:,-1]

preds = GaussianNB().fit(x_train, y_train).predict(x_test)
f1_score(y_test, preds)

0.8378378378378378

In [31]:
x_train

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_JJwaveAmp,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA
0,75,0,190,80,91,193,371,174,121,-16,...,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4
1,56,1,165,64,81,174,401,149,39,25,...,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8
2,54,0,172,95,138,163,386,185,102,96,...,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0
3,55,0,175,94,100,202,380,179,143,28,...,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6
4,75,0,190,80,88,181,360,177,103,-16,...,-0.4,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,61,1,156,77,147,147,509,327,68,31,...,-1.2,0.0,5.1,0.0,0.0,0.0,0.6,-1.9,30.6,-2.4
396,45,1,158,65,82,122,336,174,63,38,...,0.2,-0.5,7.8,-0.7,0.0,0.0,-0.1,2.5,17.6,40.6
397,30,1,160,52,110,154,426,154,94,102,...,0.5,0.0,6.5,-2.4,0.0,0.0,0.5,2.8,15.2,40.9
398,47,0,175,94,108,173,341,168,99,-49,...,-0.1,0.0,7.7,-4.0,0.0,0.0,0.9,1.5,2.5,20.2


In [40]:
accuracies = []
aucs = []

wrange = range(100,410,10)

# Loop over window sizes
for w_size in wrange:
    # Define sliding window
    t_start = t_now - w_size
    sliding = data_past.loc[t_start:t_now]
    
    # Extract X and y from the sliding window
    x, y = sliding.iloc[:,:-1], sliding.iloc[:,-1]
    
    # Fit the classifier and store the F1 score
    preds = GaussianNB().fit(x, y).predict(x_test)
    accuracies.append(f1_score(y_test, preds))
    aucs.append(roc_auc_score(y_test, preds))

# Estimate the best performing window size
optimal_window_1 = wrange[np.argmax(accuracies)]
optimal_window_2 = wrange[np.argmax(aucs)]

print(optimal_window_1, optimal_window_2)

120 190


In [41]:
accuracies

[0.8333333333333333,
 0.8333333333333333,
 0.8493150684931509,
 0.8493150684931509,
 0.8493150684931509,
 0.8493150684931509,
 0.8493150684931509,
 0.8493150684931509,
 0.8493150684931509,
 0.8450704225352113,
 0.8450704225352113,
 0.8450704225352113,
 0.8450704225352113,
 0.8333333333333333,
 0.8333333333333333,
 0.821917808219178,
 0.821917808219178,
 0.821917808219178,
 0.821917808219178,
 0.8333333333333333,
 0.821917808219178,
 0.8333333333333333,
 0.821917808219178,
 0.821917808219178,
 0.821917808219178,
 0.821917808219178,
 0.8378378378378378,
 0.8378378378378378,
 0.8266666666666667,
 0.8378378378378378,
 0.8378378378378378]

In [43]:
aucs

[0.7177033492822967,
 0.7177033492822967,
 0.7328548644338119,
 0.7328548644338119,
 0.7328548644338119,
 0.7328548644338119,
 0.7328548644338119,
 0.7328548644338119,
 0.7328548644338119,
 0.7440191387559809,
 0.7440191387559809,
 0.7440191387559809,
 0.7440191387559809,
 0.7177033492822967,
 0.7177033492822967,
 0.6913875598086124,
 0.6913875598086124,
 0.6913875598086124,
 0.6913875598086124,
 0.7177033492822967,
 0.6913875598086124,
 0.7177033492822967,
 0.6913875598086124,
 0.6913875598086124,
 0.6913875598086124,
 0.6913875598086124,
 0.7065390749601277,
 0.7065390749601277,
 0.6802232854864434,
 0.7065390749601277,
 0.7065390749601277]

## Conclusion:
### It seems a window size of 120-200 is the optimal training data size